In [1]:
!pip install mediapipe opencv-python

  Using cached matplotlib-3.7.4-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
     ---------------------------------------- 0.0/904.4 kB ? eta -:--:--
     ---- --------------------------------- 112.6/904.4 kB 2.2 MB/s eta 0:00:01
     ---------------------- --------------- 532.5/904.4 kB 5.6 MB/s eta 0:00:01
     -------------------------------------- 904.4/904.4 kB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/199.7 kB ? eta -:--:--
     ---------------------------------------- 199.7/199.7 kB ? eta 0:00:00
  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl.metadata (5.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.47.2-cp38-cp38-win_amd64.whl.metadata (160 kB)
  Using cached kiwisolver-1.4.5-cp38-cp38-win_amd64.whl.metadata (6.5 kB)
  Using cached pillow-10.2.0-cp38-cp38-win_amd64.whl.metadata (9.9 kB)
  Using cached pyparsing-3.1.1-py3-none-any.whl.metadata (5.1 kB)
   ----------------------------------------

In [ ]:
lol = []
setattr(lol, 'pose_landmarks', 'lol')
lol

In [33]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# open picture and represent it as an array
frame_sample = cv2.imread('../images/sample_003.jpg')
# downscale the frame to 480x640
frame_sample = cv2.resize(frame_sample, (640, 480))
print(type(frame_sample), frame_sample.shape)

<class 'numpy.ndarray'> (480, 640, 3)


# Make detection

In [34]:
import drawing_utils_custom as cusDU

cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame_sample, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmars
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass

        # Render detections
        cusDU.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [14]:
results.pose_landmarks

'lol'

In [16]:
len(landmarks), type(landmarks), landmarks

(33,
 google.protobuf.pyext._message.RepeatedCompositeContainer,
 [x: 0.726093590259552
 y: 0.6525042057037354
 z: -0.6397758722305298
 visibility: 0.9992882013320923
 , x: 0.7486752867698669
 y: 0.6080146431922913
 z: -0.5429905652999878
 visibility: 0.9983070492744446
 , x: 0.7632274627685547
 y: 0.614676833152771
 z: -0.5425390601158142
 visibility: 0.9967759847640991
 , x: 0.7776963114738464
 y: 0.6215528249740601
 z: -0.5425680875778198
 visibility: 0.9973766207695007
 , x: 0.7004060745239258
 y: 0.5907191038131714
 z: -0.5790209174156189
 visibility: 0.9991148710250854
 , x: 0.6790408492088318
 y: 0.5856251120567322
 z: -0.5787478089332581
 visibility: 0.9991275668144226
 , x: 0.6572329998016357
 y: 0.5813436508178711
 z: -0.5795068144798279
 visibility: 0.9996441602706909
 , x: 0.7763915657997131
 y: 0.651668131351471
 z: -0.08048124611377716
 visibility: 0.9962092041969299
 , x: 0.6059779524803162
 y: 0.6006619334220886
 z: -0.2192179262638092
 visibility: 0.9998436570167542
 ,

In [26]:
land_list = []
for i in range(len(landmarks)):
    land_list.append(landmarks[i])

In [30]:
land_list[0]

x: 0.726093590259552
y: 0.6525042057037354
z: -0.6397758722305298
visibility: 0.9992882013320923

In [49]:
def generate_landmarks(image_path):
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        # open picture and represent it as an array
        image_frame = cv2.imread(image_path)
        image_frame = cv2.resize(image_frame, (480, 480))
        image_frame = cv2.cvtColor(image_frame, cv2.COLOR_BGR2RGB)
        image_frame.flags.writeable = False
    
        # Make detection
        processed = pose.process(image_frame)
    
        # Extract landmarks
        try:
            landmark = processed.pose_landmarks.landmark
        except:
            print("---------------------LANDMARK NOT DETECTED ERROR----------------------")
            return -1
    
        landmark_list = []
        delete_list = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 17, 18, 19, 20, 21, 22, 29, 30, 31, 32]
        for i in range(len(landmark)):
            if i not in delete_list:
                if landmark[i].visibility > 0.6:
                    landmark_list.append([landmark[i].x, landmark[i].y, landmark[i].z])
                else:
                    landmark_list.append([0, 0, 0])
            
        land_arr = np.array(landmark_list, dtype=np.float64)
        return land_arr

In [50]:
arr = None
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
arr = generate_landmarks('../images/sample_003.jpg')
print(type(arr), arr.shape, arr)

<class 'numpy.ndarray'> (13, 3) [[ 0.71672636  0.64306247 -0.58199513]
 [ 0.87037188  0.89536077  0.20988449]
 [ 0.43634316  0.81642205 -0.22584482]
 [ 0.          0.          0.        ]
 [ 0.22611794  0.64119053 -0.85175544]
 [ 0.          0.          0.        ]
 [ 0.50738931  0.32446945 -1.05297244]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]


In [51]:
import landmark_utils as lu
arr2 = None
# with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
arr2 = lu.generate_landmarks('../images/sample_003.jpg')
print(type(arr2), arr2.shape, arr2)

<class 'numpy.ndarray'> (13, 3) [[ 0.71672636  0.64306247 -0.58199513]
 [ 0.87037188  0.89536077  0.20988449]
 [ 0.43634316  0.81642205 -0.22584482]
 [ 0.          0.          0.        ]
 [ 0.22611794  0.64119053 -0.85175544]
 [ 0.          0.          0.        ]
 [ 0.50738931  0.32446945 -1.05297244]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
